In [ ]:
!pip install --quiet langchain faiss-cpu tiktoken openai pinecone-client pinecone_text PyMuPDF
!pip install --upgrade transformers
!pip install git+https://github.com/naver/splade.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 33.2 MB/s eta 0:00:00
  Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
splade 2.1 requires transformers==4.18.0, but you have transformers 4.33.1 which is incompatible.


  Cloning https://github.com/naver/splade.git to /tmp/pip-req-build-gvr4h5hv
  Running command git clone --filter=blob:none --quiet https://github.com/naver/splade.git /tmp/pip-req-build-gvr4h5hv
  Resolved https://github.com/naver/splade.git to commit 3781228d5f07e7a6ae14a479e469a715de79e976
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.1
    Uninstalling transformers-4.33.1:
      Successfully uninstalled transformers-4.33.1


In [23]:
# from google.colab import drive
# drive.mount("/content/drive")

# %cd /content/drive/MyDrive/LLM/qa_rag/ulcerative_colitis/

import os, json
from config import MAIN_DIR

%cd ..
print("Current Directory:", MAIN_DIR)

/mnt/c/Users/QUAN/Desktop/medical-chatbot
Current Directory: /mnt/c/Users/QUAN/Desktop/medical-chatbot


In [2]:
import numpy as np
import pandas as pd
import torch
from splade.models.transformer_rep import Splade
from transformers import AutoModelForMaskedLM, AutoTokenizer

import sys
sys.path.append("./src")

/mnt/c/Users/QUAN/Desktop/medical-chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all.txt"), "r") as f:
    test_cases = f.readlines()

print("Total number of test cases:", len(test_cases))

with open(os.path.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    api_keys = json.load(f)

Total number of test cases: 30


# OpenAI

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.base import Embeddings

In [ ]:
EMB_DIR = os.path.join(MAIN_DIR, "data", "emb_store", "uc", "faiss", "text-embedding-ada-002")

os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]

with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all_emb.json"), "r") as f:
    test_cases_emb = json.load(f)

test_cases = [(txt, emb) for txt, emb in zip(test_cases, test_cases_emb)]
print("Number of test cases:", len(test_cases))
print("Length of embeddings:", len(test_cases[0][1]))

Number of test cases: 30
Length of embeddings: 1536


In [ ]:
embedding_function = OpenAIEmbeddings().embed_query
docsearch = FAISS.load_local(os.path.join(EMB_DIR, "v8-add-tables_2500_500"), OpenAIEmbeddings())

In [ ]:
k = 10

info = {"question": [], "average_score": [], "min_score": [], "max_score": []}

for idx in range(10):
    info[f"Doc {idx+1} text"] = []
    info[f"Doc {idx+1} score"] = []

for test_case in test_cases:
    info["question"].append(test_case[0])
    relevant_docs_and_scores = docsearch.similarity_search_with_score_by_vector(test_case[1], k = k)
    scores = [doc_and_score[1] for doc_and_score in relevant_docs_and_scores]
    info["average_score"].append(np.mean(scores))
    info["min_score"].append(np.min(scores))
    info["max_score"].append(np.max(scores))
    for idx, doc_and_score in enumerate(relevant_docs_and_scores):
        doc, score = doc_and_score
        info[f"Doc {idx+1} text"].append(doc.page_content)
        info[f"Doc {idx+1} score"].append(score)

df = pd.DataFrame(info)
save_folder = os.path.join(MAIN_DIR, "artifacts", "similarity-search-analysis-text-embedding-ada-002")
if not os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)
df.to_csv(os.path.join(save_folder, "summary.csv"), header=True)

In [ ]:
docstores = ["v6-add-tables_750_100", "v7-add-tables_1000_200", "v8-add-tables_2500_500", "v9-add-tables_1500_300"]

info = {"question": [test_case[0] for test_case in test_cases]}

for docstore in docstores:
    docsearch = FAISS.load_local(os.path.join(EMB_DIR, docstore),OpenAIEmbeddings())
    chunk_size = docstore.split("_")[-2]
    info[f"{chunk_size}_average_score"] = []
    info[f"{chunk_size}_min_score"] = []
    info[f"{chunk_size}_max_score"] = []

    for test_case in test_cases:
        relevant_docs_and_scores = docsearch.similarity_search_with_score_by_vector(test_case[1], k = k)
        scores = [doc_and_score[1] for doc_and_score in relevant_docs_and_scores]
        info[f"{chunk_size}_average_score"].append(np.mean(scores))
        info[f"{chunk_size}_min_score"].append(np.min(scores))
        info[f"{chunk_size}_max_score"].append(np.max(scores))

df_scores = pd.DataFrame(info)
save_folder = os.path.join(MAIN_DIR, "artifacts", "similarity-search-analysis-text-embedding-ada-002")
if not os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

df_scores.to_csv(os.path.join(save_folder, "compare_database.csv"), header=True)

# SPLADE

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [8]:
model_name = 'naver/splade-cocondenser-ensembledistil'
model_path = os.path.join(MAIN_DIR, "pretrained", "splade-cocondenser-ensembledistil")

if not os.path.exists(model_path):
    print(f"Cannot find model at path {model_path}. Download new files.")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

sample_testcase = test_cases[0]
print(sample_testcase)

40 year old male with newly diagnosed moderate UC and articular extraintestinal manifestations



In [13]:
splade_model = Splade(model_type_or_dir = model_path, agg="max")
splade_model.to(device)
splade_model.eval()
tokenizer = splade_model.transformer_rep.tokenizer

with torch.no_grad():
    tokens = tokenizer(sample_testcase, return_tensors = "pt")
    print(tokens)
    tokens = tokens.to(device)
    sparse_emb = splade_model(d_kwargs = tokens)
    vec = sparse_emb["d_rep"].cpu()

vec = vec[0]

{'input_ids': tensor([[  101,  2871,  2095,  2214,  3287,  2007,  4397, 11441,  8777, 15384,
          1998,  2396, 21412,  4469, 18447, 19126, 24491,  2015,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
non_zero_idx = vec.nonzero().squeeze().tolist()
print("Number of non zero tokens: {}\n".format(len(non_zero_idx)))

non_zero_weights = vec[non_zero_idx].tolist()
sparse_dict = dict(zip(non_zero_idx, non_zero_weights))

idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}

sparse_dict_tokens = {idx2token[idx]:round(weight,3) for idx, weight in sparse_dict.items()}

sparse_dict_tokens = {
    k: v for k, v in
    sorted(sparse_dict_tokens.items(), key=lambda item: item[1], reverse=True)
}

Number of non zero tokens: 51



# Hybrid Search

## SPLADE

In [16]:
import pinecone
from langchain.embeddings import OpenAIEmbeddings
from splade.models.transformer_rep import Splade
from transformers import AutoModelForMaskedLM, AutoTokenizer
import pandas as pd

In [17]:
index_name = "uc-hybrid-search"

pinecone.init(
    api_key=api_keys["PINECONE_API_SPLADE"]["KEY"],
    environment=api_keys["PINECONE_API_SPLADE"]["ENV"]
    )

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name, dimension=1536, metric="dotproduct", pod_type="s1", metadata_config={"indexed": []},
    )

In [18]:
from pinecone_text.sparse import SparseVector
from pinecone_text.sparse.base_sparse_encoder import BaseSparseEncoder
from typing import Union, List, Literal, Optional

class SpladeEncoder(BaseSparseEncoder):

    """
    SPLADE sparse vector encoder.
    Currently only supports inference with  naver/splade-cocondenser-ensembledistil
    """

    def __init__(self, model_path: Optional[str] = None, max_seq_length: int = 256,
                 agg: Literal["max", "sum"] = "max", device: str = "cpu"):
        """
        Args:
            max_seq_length: Maximum sequence length for the model. Must be between 1 and 512.
            device: Device to use for inference.

        Example:

            ```python
            from pinecone_text.sparse import SPLADE

            splade = SPLADE()

            splade.encode_documents("this is a document") # [{"indices": [102, 18, 12, ...], "values": [0.21, 0.38, 0.15, ...]}, ...]
            ```
        """
        if not 0 < max_seq_length <= 512:
            raise ValueError("max_seq_length must be between 1 and 512")

        model = model_path if model_path else "naver/splade-cocondenser-ensembledistil"

        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForMaskedLM.from_pretrained(model).to(device)
        self.max_seq_length = max_seq_length
        self.agg_fn = getattr(torch, agg)
        self.device = device

    def encode_documents(
        self, texts: Union[str, List[str]]
    ) -> Union[SparseVector, List[SparseVector]]:
        """
        encode documents to a sparse vector (for upsert to pinecone)

        Args:
            texts: a single or list of documents to encode as a string
        """
        return self._encode(texts)

    def encode_queries(
        self, texts: Union[str, List[str]]
    ) -> Union[SparseVector, List[SparseVector]]:
        """
        encode queries to a sparse vector (for upsert to pinecone)

        Args:
            texts: a single or list of queries to encode as a string
        """
        return self._encode(texts)

    def _encode(
        self, texts: Union[str, List[str]]
    ) -> Union[SparseVector, List[SparseVector]]:
        """
        Args:
            texts: single or list of texts to encode.

        Returns a list of Splade sparse vectors, one for each input text.
        """
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=self.max_seq_length,
        ).to(self.device)
        with torch.no_grad():
            logits = self.model(**inputs).logits

        inter = torch.log1p(torch.relu(logits))
        token_max = self.agg_fn(inter, dim=1)

        nz_tokens_i, nz_tokens_j = torch.where(token_max.values > 0)

        output = []
        for i in range(token_max.values.shape[0]):
            nz_tokens = nz_tokens_j[nz_tokens_i == i]
            nz_weights = token_max.values[i, nz_tokens]
            output.append(
                {"indices": nz_tokens.tolist(), "values": nz_weights.tolist()}
            )

        return output[0] if isinstance(texts, str) else output

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lequan2902/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
from langchain.retrievers import PineconeHybridSearchRetriever
from utils import convert_csv_to_documents, load_documents

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

splade_encoder = SpladeEncoder(model_path = model_path, max_seq_length = 512, agg = "max", device = device)
embeddings = OpenAIEmbeddings(openai_api_key = api_keys["OPENAI_API_KEY"])
index = pinecone.Index(index_name)

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_directory = os.path.join(MAIN_DIR, "data", "document_store", "uc")
additional_docs = os.path.join(MAIN_DIR, "data", "additional_docs.json")
exclude_pages = {
    "agrawal.pdf": [13, 14, 15, 16, 17, 18],
    "PIIS1542356520300446.pdf": [12, 13, 14, 15, 16, 17, 18],
    "gutjnl-2021-326390R2 CLEAN.pdf": [0, 2, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45],
    "otad009.pdf": [15, 16],
    "1-s2.0-S2468125321003770-main.pdf": [9],
    "juillerat 2022.pdf": [6, 7, 8],
}

chunk_size = 1000
chunk_overlap = 200

print(f"Loading documents from {source_directory}")

documents = load_documents(source_directory, exclude_pages=exclude_pages)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_documents(documents)

print(f"Loaded {len(documents)} documents from {source_directory}")
print(f"Split into {len(texts)} chunks of text (max. {chunk_size} characters each)")

with open(additional_docs, "r") as f:
    add_doc_infos = json.load(f)
for add_doc_info in add_doc_infos:
    texts.extend(convert_csv_to_documents(add_doc_info, concatenate_rows=True))

Loading documents from /mnt/c/Users/QUAN/Desktop/medical-chatbot/data/document_store/uc


Loaded 86 documents from /mnt/c/Users/QUAN/Desktop/medical-chatbot/data/document_store/uc
Split into 443 chunks of text (max. 1000 characters each)


In [25]:
page_contents = [text.page_content for text in texts]
metadatas = [text.metadata for text in texts]

retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=splade_encoder,
    index = index,
    top_k = 10,
    alpha = 0.5
    )

## retriever.add_texts(texts=page_contents, metadatas=metadatas)

In [26]:
df_list = []
k = 10

for alpha in range(11):
    retriever = PineconeHybridSearchRetriever(
        embeddings=embeddings,
        sparse_encoder=splade_encoder,
        index=index,
        top_k=k,
        alpha=alpha / 10
        )

    answer_dict = {"test_case": test_cases}
    for idx in range(k):
        answer_dict[f"doc{idx+1}"] = []

    for test_case in test_cases:
        docs = retriever.get_relevant_documents(test_case)
        for idx in range(k):
            answer_dict[f"doc{idx+1}"].append(docs[idx].page_content)

    df_list.append(pd.DataFrame(answer_dict))

In [ ]:
with pd.ExcelWriter(os.path.join(MAIN_DIR, "log", "retrieval_analysis.xlsx")) as writer:
    for idx, df in enumerate(df_list):
        df.to_excel(writer, sheet_name=f'alpha={idx/10}')

In [ ]:
for idx, df in enumerate(df_list):
    df["case_no"] = list(range(1,31))
    df["alpha"] = idx/10

combined_df = pd.concat(df_list)
combined_df = combined_df.sort_values("case_no")
combined_df.to_csv(os.path.join(MAIN_DIR, "log", "combined_retrieval_analysis.csv"))

## BM25

In [ ]:
index_name_bm25 = "uc-hybrid-search-bm25"

pinecone.init(
    api_key=api_keys["PINECONE_API2"]["KEY"],
    environment=api_keys["PINECONE_API2"]["ENV"]
    )

if index_name_bm25 not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name_bm25, dimension=1536, metric="dotproduct", pod_type="s1", metadata_config={"indexed": []},
    )

index_bm25 = pinecone.Index(index_name_bm25)

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()

# fit tf-idf values on your corpus
bm25_encoder.fit(page_contents)

# store the values to a json file
artifact_dir = os.path.join(MAIN_DIR, "artifacts")
if not os.path.exists(artifact_dir):
    os.makedirs(artifact_dir, exist_ok = True)
bm25_encoder.dump(os.path.join(artifact_dir, "bm25_values.json"))

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load(os.path.join(artifact_dir, "bm25_values.json"))

bm25_retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=bm25_encoder,
    index=index_bm25, top_k = 10, alpha = 0.5
)

# bm25_retriever.add_texts(texts=page_contents, metadatas=metadatas)

  0%|          | 0/446 [00:00<?, ?it/s]

In [ ]:
df_list = []
k = 10

for alpha in range(11):

    bm25_retriever = PineconeHybridSearchRetriever(
        embeddings=embeddings, sparse_encoder=bm25_encoder,
        index=index_bm25, top_k=k, alpha=alpha/10
    )

    answer_dict = {"test_case": test_cases}
    for idx in range(k):
        answer_dict[f"doc{idx+1}"] = []

    for test_case in test_cases:
        docs = bm25_retriever.get_relevant_documents(test_case)
        for idx in range(k):
            answer_dict[f"doc{idx+1}"].append(docs[idx].page_content)

    df_list.append(pd.DataFrame(answer_dict))

In [ ]:
for idx, df in enumerate(df_list):
    df["case_no"] = list(range(1,31))
    df["alpha"] = idx/10

combined_df = pd.concat(df_list)
combined_df = combined_df.sort_values("case_no")
combined_df.to_csv(os.path.join(MAIN_DIR, "log", "combined_retrieval_analysis_bm25.csv"))